In [1]:
import os
from dipy.align.reslice import reslice
import numpy as np
import nibabel as nb

In [3]:
def resample_nifti(nifti, 
                   order,
                   mode, #'nearest' or 'constant' or 'reflect' or 'wrap'    
                   cval,
                   in_plane_resolution_mm=1.25,
                   slice_thickness_mm=None,
                   number_of_slices=None):
    
    # sometimes dicom to nifti programs don't define affine correctly.
    resolution = np.array(nifti.header.get_zooms()[:3] + (1,))
    if (np.abs(nifti.affine)==np.identity(4)).all():
        nifti.set_sform(nifti.affine*resolution)


    data   = nifti.get_fdata().copy()
    shape  = nifti.shape[:3]
    affine = nifti.affine.copy()
    zooms  = nifti.header.get_zooms()[:3] 

    if number_of_slices is not None:
        new_zooms = (in_plane_resolution_mm,
                     in_plane_resolution_mm,
                     (zooms[2] * shape[2]) / number_of_slices)
    elif slice_thickness_mm is not None:
        new_zooms = (in_plane_resolution_mm,
                     in_plane_resolution_mm,
                     slice_thickness_mm)            
    else:
        new_zooms = (in_plane_resolution_mm,
                     in_plane_resolution_mm,
                     zooms[2])

    new_zooms = np.array(new_zooms)
    for i, (n_i, res_i, res_new_i) in enumerate(zip(shape, zooms, new_zooms)):
        n_new_i = (n_i * res_i) / res_new_i
        # to avoid rounding ambiguities
        if (n_new_i  % 1) == 0.5: 
            new_zooms[i] -= 0.001

    data_resampled, affine_resampled = reslice(data, affine, zooms, new_zooms, order=order, mode=mode , cval = cval)
    nifti_resampled = nb.Nifti1Image(data_resampled, affine_resampled)

    x=nifti_resampled.header.get_zooms()[:3]
    y=new_zooms
    if not np.allclose(x,y, rtol=1e-02):
        print('not all close: ', x,y)

    return nifti_resampled       


In [4]:
def crop_or_pad(array, target, value):
    # Pad each axis to at least the target.

    margin = target - np.array(array.shape)
    padding = [(0, max(x, 0)) for x in margin]
    array = np.pad(array, padding, mode="constant", constant_values=value)
    for i, x in enumerate(margin):
        array = np.roll(array, shift=+(x // 2), axis=i)

    if type(target) == int:
        target = [target] * array.ndim

    ind = tuple([slice(0, t) for t in target])
    return array[ind]

# function: find all files under the name * in the main folder, put theminto a file list
def find_all_target_files(target_file_name, main_folder):
    F = np.array([])
    for i in target_file_name:
        f = np.array(sorted(glob.glob(os.path.join(main_folder, os.path.normpath(i)))))
        F = np.concatenate((F, f))
    return F

In [6]:
for j in range(1,11):
    for i in range(0,10):
    
        # main
        # image file
        file = os.path.join(f'/Users/liushixing/Desktop/4DCT/DIR-LAB/Case{j}/original_image/case{j}_T{i}0_s.nii.gz')
        # nibabel load
        nii_img = nb.load(file)
    
        # get current pixel dimension
        pixdim = nii_img.header.get_zooms()
        print('current pixel dimension:', pixdim)
        print('current image dimension:', nii_img.shape)
    
        # resample to some expected dimension
        # in_plane = 1.25mm
        # slice_thickness = 2.5mm
        new_dim = [1.25,1.25,2.5]
        nii_img_resampled = resample_nifti(nii_img, order=3, mode='nearest', cval=np.min(nii_img.get_fdata()), in_plane_resolution_mm=new_dim[0], slice_thickness_mm=new_dim[-1])
    
        # turn image from float to int
        data_resampled = nii_img_resampled.get_fdata()
        data_resampled = np.round(data_resampled).astype(np.int16)
        nii_img_resampled = nb.Nifti1Image(data_resampled, nii_img_resampled.affine, nii_img_resampled.header)
    
        # save resampled image
        output_folder = f'/Users/liushixing/Desktop/4DCT/DIR-LAB/Case{j}/resampled_image'
        out_file = os.path.join(output_folder, f'case{j}_T{i}0_s.nii.gz')
        nb.save(nii_img_resampled, out_file)

        # now if you load the resampled image, you will see the new pixel dimension
        nii_img_resampled2 = nb.load(out_file)
        pixdim2 = nii_img_resampled2.header.get_zooms()
        print('new pixel dimension:', pixdim2)
        print('new image dimension:', nii_img_resampled2.shape)
    
        #now crop or pad your data into [256x256x128]
        final_dim = [256,256,128]
        data = nb.load(out_file).get_fdata()
        data_cropped_padded = crop_or_pad(data, final_dim, value=np.min(data))
        print('after crop or pad:', data_cropped_padded.shape)
        # make sure it's in int
        data_cropped_padded = np.round(data_cropped_padded).astype(np.int16)
    
        #nii_img_final = nb.Nifti1Image(data_cropped_padded, nii_img_resampled2.affine, nii_img_resampled2.header)
        nii_img_final = nb.Nifti1Image(data_cropped_padded, nii_img_resampled2.affine, nii_img_resampled2.header)
        out_file_final = os.path.join(f'/Users/liushixing/Desktop/4DCT/Popi/Case{j}/cropped_image/img_{i}.nii.gz')
        nb.save(nii_img_final, out_file_final)
print("All done!")

current pixel dimension: (np.float32(0.9765625), np.float32(0.9765625), np.float32(2.0))
current image dimension: (512, 512, 141)
new pixel dimension: (np.float32(1.25), np.float32(1.25), np.float32(2.5))
new image dimension: (400, 400, 113)
after crop or pad: (256, 256, 128)
current pixel dimension: (np.float32(0.9765625), np.float32(0.9765625), np.float32(2.0))
current image dimension: (512, 512, 141)
new pixel dimension: (np.float32(1.25), np.float32(1.25), np.float32(2.5))
new image dimension: (400, 400, 113)
after crop or pad: (256, 256, 128)
current pixel dimension: (np.float32(0.9765625), np.float32(0.9765625), np.float32(2.0))
current image dimension: (512, 512, 141)
new pixel dimension: (np.float32(1.25), np.float32(1.25), np.float32(2.5))
new image dimension: (400, 400, 113)
after crop or pad: (256, 256, 128)
current pixel dimension: (np.float32(0.9765625), np.float32(0.9765625), np.float32(2.0))
current image dimension: (512, 512, 141)
new pixel dimension: (np.float32(1.25),